In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [15]:
df = pd.read_csv('weight_encoded_2.csv')
df.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
81,0,20.000000,1.610000,64.000000,1,0,3.0,3.000000,0,0,2.000000,1,0.00000,1.000000,1,3,1
367,0,18.000000,1.600000,55.000000,0,1,2.0,4.000000,1,0,2.000000,0,2.00000,1.000000,2,3,1
896,0,22.154854,1.481682,61.373868,1,0,2.0,2.983297,2,0,1.453626,0,0.32896,0.509396,3,3,3
408,0,18.000000,1.640000,56.000000,1,1,3.0,3.000000,1,0,1.000000,1,1.00000,1.000000,3,3,1
924,1,22.000000,1.691303,80.539000,1,1,2.0,2.038373,2,0,2.000000,0,2.70825,1.506576,2,3,3


In [16]:
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

In [17]:
categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'MTRANS']
preprocessor = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'
)

In [18]:
X = preprocessor.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

In [20]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
tf.random.set_seed(42)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

In [23]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

In [24]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
28/28 [==============================] - 4s 31ms/step - loss: 1.0424 - accuracy: 0.5863 - val_loss: 0.7501 - val_accuracy: 0.7220
Epoch 2/100
28/28 [==============================] - 0s 17ms/step - loss: 0.5829 - accuracy: 0.7960 - val_loss: 0.5188 - val_accuracy: 0.8161
Epoch 3/100
28/28 [==============================] - 0s 16ms/step - loss: 0.3963 - accuracy: 0.8576 - val_loss: 0.3838 - val_accuracy: 0.8700
Epoch 4/100
28/28 [==============================] - 0s 12ms/step - loss: 0.2678 - accuracy: 0.9092 - val_loss: 0.3795 - val_accuracy: 0.8565
Epoch 5/100
28/28 [==============================] - 0s 8ms/step - loss: 0.1965 - accuracy: 0.9473 - val_loss: 0.2832 - val_accuracy: 0.9193
Epoch 6/100
28/28 [==============================] - 0s 9ms/step - loss: 0.1266 - accuracy: 0.9709 - val_loss: 0.2642 - val_accuracy: 0.9148
Epoch 7/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0838 - accuracy: 0.9843 - val_loss: 0.2407 - val_accuracy: 0.9283
Epoch 8/1

In [25]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.3434 - accuracy: 0.9462


In [26]:
# Print the evaluation results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.34339678287506104
Test Accuracy: 0.9461883306503296


In [27]:
model.save('BMI.h5')

In [28]:
from google.colab import files
files.download('BMI.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>